In [1]:
import pandas as pd
from sqlalchemy import create_engine, URL
from tqdm import tqdm

In [2]:
url_object = URL.create(
    drivername='postgresql+psycopg2',
    username='tie',
    password='TIE%2023!tuhh',
    host='134.28.58.100',
    port=45432,
    database='openalex_db',
)

In [3]:
engine = create_engine(url_object)
# Check if connection is successful
engine.connect()

In [4]:
df = pd.read_csv('/mnt/hdd01/patentsview/Reliance on Science - Cleantech Patents/df_oaid_Cleantech_Y02.csv')

In [5]:
# Convert column oaid and patent_id to string
df['oaid'] = df['oaid'].astype(str)
df['patent_id'] = df['patent_id'].astype(str)

In [6]:
# Erase everything in oaid and patent_id after the dot
df['oaid'] = df['oaid'].str.split('.').str[0]
df['patent_id'] = df['patent_id'].str.split('.').str[0]

In [7]:
# Add column 'full_oaid' with https://openalex.org/W + oaid
df['full_oaid'] = 'https://openalex.org/W' + df['oaid']

In [8]:
# Keep only columns oaid, patent_id and full_oaid
df = df[['oaid', 'patent_id', 'full_oaid']]

In [9]:
# Insert temporary table into database
df.to_sql('temp_table', engine, if_exists='replace', index=False)

145

In [10]:
# SQL query to get all data from data table that has an id in the temp_table
query = '''
    SELECT *
    FROM openalex.works 
    JOIN temp_table ON openalex.works.id = temp_table.full_oaid
'''

In [11]:
result = pd.read_sql(query, engine)

In [12]:
print(result.columns)

Index(['id', 'doi', 'title', 'display_name', 'publication_year',
       'publication_date', 'type', 'cited_by_count', 'is_retracted',
       'is_paratext', 'cited_by_api_url', 'abstract_inverted_index',
       'abstract', 'oaid', 'patent_id', 'full_oaid'],
      dtype='object')


In [13]:
print(result['abstract_inverted_index'].head())

0    {'IndexLength': 254, 'InvertedIndex': {'Severa...
1    {'IndexLength': 29, 'InvertedIndex': {'In': [0...
2    {'IndexLength': 197, 'InvertedIndex': {'The': ...
3    {'IndexLength': 255, 'InvertedIndex': {'In': [...
4    {'IndexLength': 17, 'InvertedIndex': {'As': [0...
Name: abstract_inverted_index, dtype: object


In [14]:
# Delete all empty rows in column abstract_inverted_index with value None
result = result[result['abstract_inverted_index'].notna()]

In [15]:
print(result['abstract_inverted_index'].iloc[0])

{'IndexLength': 254, 'InvertedIndex': {'Several': [0], 'studies': [1, 173, 226], 'have': [2], 'described': [3], 'a': [4, 135, 182], 'clear': [5], 'association': [6], 'between': [7], 'respiratory': [8, 13, 93, 157], 'syncytial': [9], 'virus': [10], '(RSV)': [11], 'lower': [12, 156], 'tract': [14, 94, 158], 'infection': [15], 'in': [16, 25, 91, 154, 188, 237, 245, 252], 'infancy': [17], 'and': [18, 44, 95, 126, 210, 241], 'the': [19, 28, 57, 73, 80, 92, 106, 149, 155, 231, 238, 246], 'subsequent': [20], 'development': [21], 'of': [22, 59, 75, 151, 159, 163, 184, 201, 212, 233, 248], 'persistent': [23, 127, 197, 234, 250], 'wheezing': [24, 251], 'children.': [26, 253], 'Using': [27], 'mouse': [29, 239], 'model': [30, 51], 'we': [31, 96], 'demonstrated': [32, 175], 'that': [33, 176], 'RSV': [34, 49, 89, 111, 133, 139, 152, 213, 235], 'induces': [35], 'long-term': [36, 61, 219], 'airway': [37, 42, 45, 62, 121, 128, 220], 'disease': [38, 122, 221], 'characterized': [39, 123], 'by': [40, 110,

In [16]:
print(len(result))

597001


In [35]:
result['abstract'] = ''
# Iterate over abstract_inverted_index column
for index, row in tqdm(result.iterrows()):
    word_index = []
    count += 1
    try:
        for key, value in row['abstract_inverted_index'].items():
            if key == 'InvertedIndex':
                for innerkey, innervalue in value.items():
                    for innerindex in innervalue:
                        word_index.append([innerkey, innerindex])
        # Sort list by index
        word_index.sort(key=lambda x: x[1])
        # Join first element of each list in word_index
        abstract = ' '.join([i[0] for i in word_index])
        # Add column abstract to result dataframe
        # result.loc[index, 'abstract'] = abstract
        result.at[index, 'abstract'] = abstract
        print(index, abstract)
    except AttributeError:
        continue

4it [00:00, 10.00it/s]

0 Several studies have described a clear association between respiratory syncytial virus (RSV) lower respiratory tract infection in infancy and the subsequent development of persistent wheezing in children. Using the mouse model we demonstrated that RSV induces long-term airway disease characterized by chronic airway inflammation and airway hyperreactivity (AHR). The RSV murine model offers great advantages to study the immunopathogenesis of RSV-induced long-term airway disease. Mice can be challenged with aerosolized methylcholine to determine the presence of AHR. We can apply the reverse transcription-polymerase chain reaction assay (RT-PCR) to detect RSV RNA in the respiratory tract and we can perform lung gene expression analysis to further characterize the chronic changes induced by RSV infection. Compared with sham-inoculated controls, RSV-infected mice developed chronic airway disease characterized by AHR and persistent airway inflammation. Forty-two days after RSV infection, a 

In [27]:
print(result['abstract'].head(50))

0                                                      
1                                                      
2                                                      
3                                                      
4                                                      
7                                                      
8                                                      
9                                                      
10                                                     
11                                                     
12                                                     
13                                                     
15                                                     
16    As the less familiar cousin of quantitative PC...
17                                                     
18                                                     
19                                                     
21                                              

In [29]:
# Save result dataframe to json
result_test = result.head(100)
result_test.to_json('/home/thiesen/Documents/Cleantech_Concepts/oaid_test.json', orient='records')

In [37]:
df_new = pd.read_json('/mnt/hdd01/patentsview/Reliance on Science - Cleantech Patents/df_oaid_Cleantech_Y02_works.json')
df_new_test = df_new.head(1000)
df_new_test.to_json('/home/thiesen/Documents/Cleantech_Concepts/df_oaid_test.json', orient='records')